In [129]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

## Part 1 : EDA and data preprocessing

In [130]:
df = pd.read_csv("Walmart_Store_sales.csv")

Let's start by exploring the dataset.

In [131]:
print(f'Number of rows: {df.shape[0]}\n')
print(f'Number of columns: {df.shape[1]}\n')
print("Display of dataset:\n")
display(df.head())
print('Basics statistics:\n')
display(df.describe(include='all'))
print('Percentage of missing values:\n')
missing_percentage = df.isnull().sum() * 100 / len(df)
display(missing_percentage)

Number of rows: 150

Number of columns: 8

Display of dataset:



,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


Basics statistics:



,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Percentage of missing values:



Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

The target variable corresponds to the column "Weekly_sales". Let's drop the rows the value is missing.

In [132]:
df = df.dropna(subset="Weekly_Sales")

Let's convert the column "Date" to datetime type and create the following columns : "year", "month", "day", "day_of_week".

In [133]:
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)

df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["day"] = df["Date"].dt.day
df["day_of_week"] = df["Date"].dt.dayofweek

df = df.drop("Date", axis=1)

We now need to drop rows where the value in the columns Temperature, Fuel_Price, CPI and Unemployment is not in [X̄ - 3σ, X̄ + 3σ].


In [135]:
columns = ["Temperature", "Fuel_Price", "CPI", "Unemployment"]
for column in columns:
    mean = df[column].mean()
    std = df[column].std()
    df = df.drop(df[(df[column] <= mean - 3*std) & (df[column] >= mean - 3*std)].index)
df.shape

(136, 11)